In [74]:
# Carrega biblioteca pandas
import pandas as pnd
# Carrega biblioteca numpy
import numpy as np
# Carrega biblioteca matplotlib
import matplotlib.pyplot as plt
# Carrega biblioteca seaborn
import seaborn as sb

from mpl_toolkits.mplot3d import Axes3D

import requests

from sklearn.model_selection import train_test_split

from sklearn.metrics import silhouette_score


from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler

from geopy.geocoders import Nominatim

from geopy.exc import GeocoderTimedOut, GeocoderQuotaExceeded

import time

In [ ]:
# Concatena diferentes tabelas com cada mês (dados)
# Em seguida salva todos os dados em um único DataFrame chamado df
listas = ['month_2.csv', 'month_3.csv', 'month_4.csv', 'month_5.csv', 'month_6.csv']
df = []
for arquivo in listas:
    df += [pnd.read_csv(arquivo)]

# Concatena todos os dataframes em um único dataframe chamado df
df = pnd.concat(df)

# Chama o dataframe contido na variável chamada df
df

In [ ]:
# Carrega e salva cadastroConsumindo como cadastroConsumindo
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')

# Chama o dataframe contido na variável dadoCadastrais
dadosCadastrais

In [ ]:
cadastroConsumindo = cadastroConsumindo.loc[cadastroConsumindo['situacao'] == "CONSUMINDO GÁS"]

print(cadastroConsumindo.shape[0], cadastroConsumindo.shape[0])

In [ ]:
# Carrega diferentes possibilidades para a coluna 'cidade'
# Antes da normalização
cadastroConsumindo['cidade'].unique()

# Substituindo "GRAVATAI" por "GRAVATAÍ" na coluna "cidade"
cadastroConsumindo['cidade'] = cadastroConsumindo['cidade'].replace('GRAVATAI', 'GRAVATAÍ')

# Substituindo "SAO LEOPOLDO" por "SÃO LEOPOLDO" na coluna "cidade"
cadastroConsumindo['cidade'] = cadastroConsumindo['cidade'].replace('SAO LEOPOLDO', 'SÃO LEOPOLDO')

# Verificando entradas na coluna "cidade"
cadastroConsumindo['cidade'].unique()

In [ ]:
# Função para obter latitude e longitude com retry
def obter_lat_long_cep(cep, tentativas=3, atraso=2):
    geolocator = Nominatim(user_agent="geoapi_cadastro")

    for tentativa in range(tentativas):
        try:
            time.sleep(1)  # Respeitar a política de uso da API
            location = geolocator.geocode(cep, timeout=10)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except Exception as e:
            print(f"Tentativa {tentativa + 1} falhou: {e}")
            if tentativa < tentativas - 1:  # Não aguarda se for a última tentativa
                time.sleep(atraso)
    
    return None, None

# Aplicar a função ao DataFrame com retry
cadastroConsumindo['latitude'], cadastroConsumindo['longitude'] = zip(*cadastroConsumindo['cep'].apply(lambda cep: obter_lat_long_cep(cep)))



In [ ]:
cadastroConsumindo['latitude'], cadastroConsumindo['longitude']


In [81]:
def obter_altitude(lat, lon):
    url = f'https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}'
    
    try:
        # Solicitação GET para a API de altitude
        resposta = requests.get(url)
        if resposta.status_code == 200:
            # Extrair a altitude da resposta JSON
            dados = resposta.json()
            altitude = dados['results'][0]['elevation']
            return altitude
        else:
            return None
    except Exception as e:
        print(f"Erro ao buscar altitude: {e}")
        return None

# Aplicar a função para obter a altitude e criar uma nova coluna 'altitude'
cadastroConsumindo['altitude'] = cadastroConsumindo.apply(lambda row: obter_altitude(row['latitude'], row['longitude']), axis=1)



In [ ]:


num_nans = cadastroConsumindo['altitude'].isna().sum()

print(num_nans)

In [ ]:


nans_latitude = cadastroConsumindo["latitude"].isna().sum()



nans_longitude = cadastroConsumindo["longitude"].isna().sum()

print(nans_latitude, nans_longitude)

In [ ]:
plt.figure(figsize=(14, 8))
plt.scatter(cadastroConsumindo['longitude'], cadastroConsumindo['altitude'], color='blue', alpha=0.5, edgecolors='w', s=10)
plt.title('Relação entre Longitude e Altitude')
plt.xlabel('Longitude')
plt.ylabel('Altitude (metros)')
plt.grid(True)
plt.show()

In [ ]:
merged_df = pnd.merge(df, cadastroConsumindo, on=['clientCode']) 

nans_clientCode = df["clientCode"].isna().sum()

print(merged_df.altitude.head(100))
print(cadastroConsumindo.altitude.head(100))

print(merged_df.shape[0])
print(df.shape[0])


In [ ]:
print(cadastroConsumindo.altitude.describe(),
merged_df.altitude.describe())


In [ ]:
merged_df.columns

print(df.shape[0], merged_df.shape[0])

In [ ]:
#Cria a variação do pulseCount como uma coluna nova, calculando por grupo a diferença
merged_df['diffPulseCount'] = merged_df.groupby(['clientCode', 'meterSN']).pulseCount.diff()
#Preenche os valores nulos (iniciais) com 0
merged_df['diffPulseCount'].fillna(0, inplace=True)

In [ ]:
print(merged_df.altitude.describe(), cadastroConsumindo.altitude.describe(), cadastroConsumindo.clientCode.describe())


In [ ]:
dadosProcessados = merged_df.dropna(subset=['diffPulseCount', 'altitude'])

dadosProcessados.shape[0]
merged_df.shape[0]

In [ ]:
num_negativos = (dadosProcessados['diffPulseCount'] < 0).sum()

print(f'Número de valores negativos: {num_negativos}')

dadosPositivos = dadosProcessados[dadosProcessados['diffPulseCount'] >= 0]

num_negativos = (dadosPositivos['diffPulseCount'] < 0).sum()

print(f'Número de valores negativos: {num_negativos}')

In [92]:
scaler = StandardScaler()
dados_scaled = scaler.fit_transform(dadosPositivos[['diffPulseCount', 'altitude']])


In [ ]:
sse = []  # Soma dos erros quadráticos (distância dos pontos ao centróide mais próximo)
k_values = range(1, 11)  # Testando de 1 a 10 clusters

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(dados_scaled)
    sse.append(kmeans.inertia_)

# Visualizando o Elbow plot
plt.figure(figsize=(8, 5))
plt.plot(k_values, sse, 'bo-')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('SSE (Soma dos Erros Quadráticos)')
plt.title('Método Elbow para Escolha do k')
plt.show()

In [ ]:
k = 3  # Defina o valor de k conforme a análise do gráfico acima
kmeans = KMeans(n_clusters=k, random_state=42)
dadosPositivos['cluster'] = kmeans.fit_predict(dados_scaled)

In [ ]:
plt.figure(figsize=(8, 6))
sb.scatterplot(x='diffPulseCount', y='altitude', hue='cluster', data=dadosPositivos, palette='viridis')
plt.xlabel('Delta Pulse Count')
plt.ylabel('Altitude')
plt.title('Clusters de Clientes por Variação de Pulse Count e Altitude')
plt.legend()
plt.show()

In [ ]:
def identify_outliers(group):
    Q1 = group['diffPulseCount'].quantile(0.25)
    Q3 = group['diffPulseCount'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Marcar outliers
    group['Outlier'] = group['diffPulseCount'].apply(
        lambda x: 'Outlier' if x < lower_bound or x > upper_bound else 'Normal'
    )
    return group

# Aplicar a função a cada cluster
df_with_outliers = dadosPositivos.groupby('cluster').apply(identify_outliers)

num_outliers = df_with_outliers['Outlier'].eq("Outlier").sum()

print(num_outliers)

In [ ]:
dadosProcessados2 = merged_df.dropna(subset=['diffPulseCount', 'altitude', 'latitude', 'longitude'])

dadosProcessados2.shape[0]

In [ ]:
num_negativos2 = (dadosProcessados2['diffPulseCount'] < 0).sum()

print(f'Número de valores negativos: {num_negativos2}')

dadosPositivos2 = dadosProcessados2[dadosProcessados2['diffPulseCount'] >= 0]

num_negativos2 = (dadosPositivos2['diffPulseCount'] < 0).sum()

print(f'Número de valores negativos: {num_negativos2}')

In [ ]:
# 1. Combinar as variáveis (latitude e longitude)
dadosPositivos2['coordinates'] = list(zip(dadosPositivos2['latitude'], dadosPositivos2['longitude']))

# 2. Preparar os dados
coordinates = dadosPositivos2['coordinates'].tolist()
coordinates_df = pnd.DataFrame(coordinates, columns=['latitude', 'longitude'])

# Concatenando com o delta_pulse_count
data_for_kmeans = pnd.concat([coordinates_df, dadosPositivos2['diffPulseCount']], axis=1).dropna()

print(data_for_kmeans.isna().sum())


In [ ]:
sse = []  # Soma dos erros quadráticos (distância dos pontos ao centróide mais próximo)
k_values = range(1, 11)  # Testando de 1 a 10 clusters

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_for_kmeans)
    sse.append(kmeans.inertia_)

# Visualizando o Elbow plot
plt.figure(figsize=(8, 5))
plt.plot(k_values, sse, 'bo-')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('SSE (Soma dos Erros Quadráticos)')
plt.title('Método Elbow para Escolha do k')
plt.show()

In [101]:
# Dividindo os dados em treino e teste
X_train, X_test = train_test_split(data_for_kmeans, test_size=0.2, random_state=42)

# 3. Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 4. Executar o K-means
kmeans = KMeans(n_clusters=3, random_state=42)  # ajuste o número de clusters conforme necessário
kmeans.fit(X_train_scaled)

# 5. Fazer previsões no conjunto de teste
y_test_pred = kmeans.predict(X_test_scaled)

# 6. Avaliar o modelo com silhouette score no conjunto de teste
silhouette_avg = silhouette_score(X_test_scaled, y_test_pred)
print(f'Silhouette Score no conjunto de teste: {silhouette_avg}')

In [ ]:
X_train['cluster'] = kmeans.labels_

X_train['latitude'].value_counts()
X_train['longitude'].value_counts()

In [ ]:
centroides = kmeans.cluster_centers_

# 2. Criar o gráfico de dispersão
plt.figure(figsize=(10, 8))

# Scatter plot dos dados com base nos clusters
sb.scatterplot(x=data_for_kmeans['longitude'], 
                y=data_for_kmeans['latitude'], 
                hue=X_train['cluster'],  # Cor baseada nos clusters
                palette='deep',      # Paleta de cores
                s=50,                # Tamanho dos pontos
                alpha=0.7)           # Transparência para melhor visualização

# 3. Adicionar os centroides no gráfico
plt.scatter(centroides[:, 1], centroides[:, 0],  # Longitude e Latitude dos centroides
            c='red', 
            s=200, 
            marker='X', 
            label='Centroides')

# 4. Títulos e legendas
plt.title('Clusters de Clientes com K-means', fontsize=14)
plt.xlabel('Longitude', fontsize=12)
plt.ylabel('Latitude', fontsize=12)
plt.legend()
plt.grid(True)

# 5. Mostrar o gráfico
plt.show()

In [ ]:
# 2. Criar o gráfico de dispersão 3D
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot dos dados, com clusters em 3D
scatter = ax.scatter(X_train['longitude'],  # eixo X
                     X_train['latitude'],   # eixo Y
                     X_train['diffPulseCount'], # eixo Z (delta pulse count)
                     c=X_train['cluster'],    # Cor com base nos clusters
                     cmap='viridis',     # Paleta de cores
                     s=50,               # Tamanho dos pontos
                     alpha=0.7)          # Transparência dos pontos

# 4. Títulos e rótulos dos eixos
ax.set_title('Clusters de Clientes com K-means (Longitude, Latitude e Delta Pulse Count)', fontsize=14)
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.set_zlabel('Delta Pulse Count', fontsize=12)

# 5. Adicionar uma barra de cores para facilitar a leitura dos clusters
cbar = fig.colorbar(scatter)
cbar.set_label('Cluster')

# 6. Mostrar o gráfico
plt.show()

In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min

# Calculando as distâncias entre os pontos e os centróides
distances = pairwise_distances_argmin_min(kmeans.cluster_centers_, X_train)[1]

# Definir um limiar de distância para considerar um ponto como anômalo
threshold = 1.5 * distances.mean()  # Exemplo de limiar
anomalias = distances > threshold

print("Número de anomalias detectadas:", sum(anomalias))
